# 해시법(hashing)
- 검색과 더불어 데이터의 추가 및 삭제도 효율적으로 가능한 방법
- 데이터를 저장할 위치(인덱스)를 구하는 연산 방법

## 해시법이란

**해시값(hash value)**
> 배열 내의 값을 원소 개수로 나눈 나머지를 해시값이라고 함


    
|**인덱스**|0|1|2|3|4|5|6|7|8|9|10|11|12|    
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|  
|**기존배열**|5|6|14|20|29|34|37|51|69|75|-|-|-|  
|**해시값**|5|6|1|7|3|8|11|12|4|10|-|-|-|  

총 13개의 원소를 가지는 배열에 10개의 값이 들어있다고 할 때, 해시값은 위와 같이 각각의 원소를 총 원소의 개수 13으로 나눈 나머지가 된다.
    

해시값이 새로운 배열의 인덱스와 같다. 아래를 보면 이해 가능.

<center>  
hash table 
<center>  

|**인덱스**|0|1|2|3|4|5|6|7|8|9|10|11|12|    
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|   
|**해시배열**|-|14|-|29|69|5|6|20|34|-|75|37|51|

설명하자면, 해시배열에서 **x[1]은 14**이다. **14%13 = 1** 이기 때문.  
같은 이유로 **34%13 = 8이기 때문에 x[8] = 34**이다.

여기서 만약 35를 새롭게 추가하고자 한다면, 해시법을 사용하여 추가할 수 있다.  
**35 % 13 = 9 이므로, x[9] = 35가 된다.**

## 해시법에서의 충돌

**충돌(collision)**
> 하나의 해시값에 대해 여러 키가 중첩되는 경우를 충돌이 발생했다고 함.

<p>
예를 들어서, 총 원소의 개수가 20인 배열에서, 원소 `1`과 `21`은 동일한 해시값 `1`을 가지므로 해시법을 사용했을 때 충돌이 발생할 수 있음.


## 충돌 대처법
1. 체인법: 해시값이 같은 원소를 연결 리스트로 관리함
2. 오픈 주소법: 빈 버킷을 찾을 때까지 해시를 반복함

### 체인법

`Node` 클래스 : 값을 담는 bucket 형태
- key, value, next 값으로 구성
- next는 다음 노드를 참조하는 단일 연결 리스트 형태

`ChainedHash` 클래스: 해시법 with 체인법 

- `hash_value(key)` : 해시값 구하기
 - key에 해당하는 해시값을 반환함  

- `search(key)` : key인 원소 검색   
 - key의 해시값을 구한다   
 - 해시값에 해당하는 Node(bucket)를 찾아 key 값이 나올때까지 탐색한다.
 - 끝까지 못찾는다면 key값이 해시 테이블 안에 없는 것임.  

- `add(key, value)` : key와 값이 value인 원소를 추가
 - key에 해당하는 해시값을 구한다
 - 해시값에 해당하는 원소에 집중하여 노드의 끝 지점을 찾는다.
 - 만약 같은 key 값이 있다면 추가 실패를, 노드의 끝 지점을 찾았다면, 추가 성공임을 반환한다.  

- `remove(key)` : 키가 key인 원소를 삭제
 - key에 해당하는 해시값을 구한다
 - 해시값에 해당하는 원소에 집중하여 key를 갖고 있는 노드를 찾는다.
 - 노드를 찾는다면 노드를 삭제하고 바로 앞의 노드와 바로 뒤의 노드를 연결하고, 성공했음을 반환한다.
 - key값이 같은 원소가 없다면 실패했음을 반환한다.  

- `dump()` : 현재의 해시 테이블 상황을 출력한다.

In [46]:
import hashlib

# 노드 클래스 생성
class Node:
    
    def __init__(self, key, value, next):
        self.key = key
        self.value = value
        self.next = next

"""체인법으로 해시 클래스 구현"""
class ChainedHash:
    
    # 초기화
    def __init__(self, length:int):
        self.length = length # 배열의 총 길이 정의
        self.table = [None] * self.length # 길이가 length인 빈 해시 테이블 생성
        
    # 해시값 구하기
    def hash_value(self, key):
        
        if isinstance(key, int): # isinstance(a, int) : a가 int형이면 True 아니면 False 반환 
            return key % self.length
        
        """
        key값은 정수가 아닐 수 있음. 이때 아래와 같은 함수로 byte문자열로 인코딩하고 16진수로 바꿔서 해시값을 구할 수 있음
        """
        
        return( int(hashlib.sha256(str(key).encode()).hexdigest(), 16) % self.length ) 
    
    
    # 삽입
    def add(self, key, value):
        """ 해시값 구하고, 해시값에 해당하는 노드에 추가함"""
        hash = self.hash_value(key)
        p = self.table[hash]
        
        while p is not None: # 비어있는 노드를 찾기 위해서 계속 탐색하는 과정
            if p.key == key:  # 키 값이 같다 = 이미 있는 동일한 값이다 -> 종료
                return False
            p = p.next
            
        temp = Node(key, value, self.table[hash])
        self.table[hash] = temp
        return True
        
    
    # 삭제
    def remove(self, key):
        hash = self.hash_value(key)
        p = self.table[hash]
        pp = None
        
        while p is not None: # 값이 존재하는 Node 에 한하여
            if p.key == key:
                if pp is None:
                    self.table[hash] = p.next
                else:
                    pp.next = p.next
                return True
            pp = p
            p = p.next
        return False
            
    # 검색
    def search(self, key):
        """ 해시값 구하고, 해시값에 해당하는 노드에서 key가 나올 때까지 탐색함. 없다면 없다고 출력 """
        hash = self.hash_value(key) # 해시값 구하기
        p = self.table[hash] # 해시값에 해당하는 노드 선택
        
        while p is not None: # 끝까지 탐색
            if p.key == key:  # 찾기 성공
                return p.value 
            p = p.next # 못찾으면 다음 노드 탐색
        
        return None # while문을 탈출 = 찾기 실패
            
        
    # 출력
    def dump(self):
        for i in range(self.length):
            p = self.table[i]
            print(i, end = "")
            while p is not None:
                print(f" -> {p.key}:{p.value}", end = "")
                p = p.next
            print()
        
        
        

- `sha256` 알고리즘: 주어진 바이트 문자열의 해시값을 구하는 해시 알고리즘의 생성자.
- `sha256`에 인풋값으로 바이트 문자열을 집어넣기 위해서 `str`자료형으로 바꾸고 `encode()`로 인코딩해줌
- `sha256`의 아웃풋(해시값)을 `hexdigest()`를 이용해 16진수로 바꿈

In [47]:
chained_hash = ChainedHash(10)
chained_hash.add(10, 8)
chained_hash.add(15,"z")
chained_hash.add(11,"t")
chained_hash.add(12,"e")
chained_hash.add(13,"f")
chained_hash.add(14,"d")
chained_hash.add(25,"c")
chained_hash.add(16,"b")
chained_hash.add(17,"a")

True

In [48]:
chained_hash.dump()

0 -> 10:8
1 -> 11:t
2 -> 12:e
3 -> 13:f
4 -> 14:d
5 -> 25:c -> 15:z
6 -> 16:b
7 -> 17:a
8
9


In [49]:
chained_hash.remove(25)

True

In [50]:
chained_hash.dump()

0 -> 10:8
1 -> 11:t
2 -> 12:e
3 -> 13:f
4 -> 14:d
5 -> 15:z
6 -> 16:b
7 -> 17:a
8
9
